# Notebook 2 - SQL Analysis
**Objectif :** Effectuer des agrégations exploratoires sur les données originales (non transformées) via Spark SQL.

In [1]:
# Étape 1 : Initialisation Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Fraud-SQL-Analysis") \
    .getOrCreate()

print(f"✅ Spark {spark.version} session created")

✅ Spark 3.5.0 session created


In [2]:
# Étape 2 : Charger le dataset CSV original et créer la vue temporaire 
# Chemin de base HDFS (hôte:port)
hdfs_base_path = "hdfs://namenode:8020/data"

# 1. Charger les données d'entraînement (Train)
# Correction de l'URI HDFS : hdfs://namenode:8020/fraud_data/train
train_df = spark.read.parquet(f"{hdfs_base_path}/train")

# 2. Charger les données de test (Test)
test_df = spark.read.parquet(f"{hdfs_base_path}/test")

# 3. Créer la vue temporaire pour l'analyse SQL (en utilisant les données d'entraînement)
train_df.createOrReplaceTempView("train_transactions")
test_df.createOrReplaceTempView("test_transactions")

# Vérification
print("Schéma du DataFrame d'entraînement Parquet :")
train_df.printSchema()
train_df.show(5)

Schéma du DataFrame d'entraînement Parquet :
root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[38648.0,1.572105...|    0|
|[38649.0,-0.26639...|    0|
|[38649.0,0.649402...|    0|
|[38650.0,0.820778...|    0|
|[38652.0,1.129032...|    0|
+--------------------+-----+
only showing top 5 rows



In [3]:
# Étape 3 : Nombre de transactions par classe 
print("--- Nombre de transactions par classe ---")
df_class_count = spark.sql("SELECT label, COUNT(*) AS count FROM train_transactions GROUP BY label")
df_class_count.show()

--- Nombre de transactions par classe ---
+-----+------+
|label| count|
+-----+------+
|    1|   400|
|    0|227645|
+-----+------+



In [4]:
# 🔹 Étape 4 : Montant moyen et Écart-type par classe 
from pyspark.ml.functions import vector_to_array # Importation de la fonction vector_to_array
from pyspark.sql.functions import col, avg, stddev

print("\n--- Montant moyen et Écart-type par classe ---")

# 1. Conversion du vecteur 'features' en tableau (Array)
df_array = train_df.withColumn(
    "features_array",
    vector_to_array(col("features"))
)

# 2. Extraction de la colonne 'Amount' du tableau
df_with_amount = df_array.withColumn(
    "Amount",
    col("features_array").getItem(29)
)

# 3. Calcul des statistiques
df_avg_amount = df_with_amount.groupBy("label").agg(
    avg(col("Amount")).alias("avg_amount"),
    stddev(col("Amount")).alias("stddev_amount")
)

# 4. Affichage des résultats
df_avg_amount.show()


--- Montant moyen et Écart-type par classe ---
+-----+------------------+------------------+
|label|        avg_amount|     stddev_amount|
+-----+------------------+------------------+
|    1|125.60019999999999|258.35134450196875|
|    0|  88.3355424454762| 249.5706177774838|
+-----+------------------+------------------+



In [9]:
# 1. INITIALISATION (comme section 1 du TP)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, sum, max, min, stddev, when

spark = SparkSession.builder \
    .appName("Fraud-GroupBy-OrderBy") \
    .getOrCreate()

# 2. CHARGEMENT DES DONNÉES (comme section 2.2 du TP)
hdfs_base_path = "hdfs://namenode:8020/data"

# Charger les données Parquet
train_df = spark.read.parquet(f"{hdfs_base_path}/train")

# 3. EXTRACTION DU MONTANT (si besoin)
from pyspark.ml.functions import vector_to_array

# Convertir le vecteur features en tableau
df_array = train_df.withColumn("features_array", vector_to_array(col("features")))

# Extraire le montant (position 29 selon ton TP)
df_with_amount = df_array.withColumn("Amount", col("features_array")[29])

# Créer une vue SQL
df_with_amount.createOrReplaceTempView("transactions")

# 4. MAINTENANT LES GROUP BY ET ORDER BY FONCTIONNERONT

In [10]:
# a) Nombre de transactions par classe
df_grouped = train_df.groupBy("label").count()
print("=== Transactions par classe ===")
df_grouped.show()

# b) Avec plusieurs agrégations
print("\n=== Statistiques par classe ===")
df_with_amount.groupBy("label") \
    .agg(
        count("*").alias("nb_transactions"),
        avg("Amount").alias("avg_amount"),
        sum("Amount").alias("total_amount"),
        max("Amount").alias("max_amount"),
        min("Amount").alias("min_amount"),
        stddev("Amount").alias("std_amount")
    ) \
    .show()

=== Transactions par classe ===
+-----+------+
|label| count|
+-----+------+
|    1|   400|
|    0|227645|
+-----+------+


=== Statistiques par classe ===
+-----+---------------+------------------+--------------------+----------+----------+------------------+
|label|nb_transactions|        avg_amount|        total_amount|max_amount|min_amount|        std_amount|
+-----+---------------+------------------+--------------------+----------+----------+------------------+
|    1|            400|125.60019999999999|  50240.079999999994|   2125.87|       0.0|258.35134450196875|
|    0|         227645|  88.3355424454762|2.0109144560000427E7|  25691.16|       0.0| 249.5706177774838|
+-----+---------------+------------------+--------------------+----------+----------+------------------+



In [11]:
# Tri ascendant
print("\n=== Tri ascendant par nombre ===")
df_grouped.orderBy("count").show()

# Tri descendant
print("\n=== Tri descendant par nombre ===")
df_grouped.orderBy(col("count").desc()).show()


=== Tri ascendant par nombre ===
+-----+------+
|label| count|
+-----+------+
|    1|   400|
|    0|227645|
+-----+------+


=== Tri descendant par nombre ===
+-----+------+
|label| count|
+-----+------+
|    0|227645|
|    1|   400|
+-----+------+



In [12]:
print("\n=== Version SQL ===")
spark.sql("""
    SELECT 
        label,
        COUNT(*) as nb_transactions,
        ROUND(AVG(Amount), 2) as avg_amount,
        ROUND(MAX(Amount), 2) as max_amount
    FROM transactions
    GROUP BY label
    ORDER BY nb_transactions DESC
""").show()


=== Version SQL ===
+-----+---------------+----------+----------+
|label|nb_transactions|avg_amount|max_amount|
+-----+---------------+----------+----------+
|    0|         227645|     88.34|  25691.16|
|    1|            400|     125.6|   2125.87|
+-----+---------------+----------+----------+



In [13]:
print("\n=== Analyse par plages de montants ===")

df_with_bins = df_with_amount.withColumn(
    "amount_range",
    when(col("Amount") < 10, "0-10")
    .when(col("Amount") < 50, "10-50")
    .when(col("Amount") < 100, "50-100")
    .when(col("Amount") < 500, "100-500")
    .when(col("Amount") < 1000, "500-1000")
    .otherwise("1000+")
)

df_with_bins.groupBy("label", "amount_range") \
    .agg(
        count("*").alias("count"),
        avg("Amount").alias("avg_in_range")
    ) \
    .orderBy("label", "amount_range") \
    .show()


=== Analyse par plages de montants ===
+-----+------------+-----+------------------+
|label|amount_range|count|      avg_in_range|
+-----+------------+-----+------------------+
|    0|        0-10|77750| 3.698326045016149|
|    0|       10-50|73973|25.126650669838863|
|    0|     100-500|38250|208.45337437908464|
|    0|       1000+| 2444|1773.9250327332243|
|    0|      50-100|30061|  71.8065796214364|
|    0|    500-1000| 5167| 676.5074588736212|
|    1|        0-10|  203| 1.731871921182266|
|    1|       10-50|   47|26.684468085106385|
|    1|     100-500|   79|238.57974683544307|
|    1|       1000+|    7|1475.8085714285714|
|    1|      50-100|   41| 87.31292682926829|
|    1|    500-1000|   23| 690.2630434782608|
+-----+------------+-----+------------------+



In [14]:
# 1. Distribution des montants par classe avec tri
df_with_amount.groupBy("label") \
    .agg(
        count("*").alias("nb"),
        avg("Amount").alias("avg"),
        stddev("Amount").alias("std")
    ) \
    .orderBy(col("avg").desc()) \
    .show()

# 2. Top 10 des montants les plus élevés par classe
df_with_amount.filter(col("label") == 1) \
    .select("Amount") \
    .orderBy(col("Amount").desc()) \
    .limit(10) \
    .show()

# 3. Nombre de transactions par plage de montants
from pyspark.sql.functions import when

df_with_bins = df_with_amount.withColumn(
    "amount_range",
    when(col("Amount") < 10, "0-10")
    .when(col("Amount") < 100, "10-100")
    .when(col("Amount") < 1000, "100-1000")
    .otherwise("1000+")
)

df_with_bins.groupBy("label", "amount_range") \
    .count() \
    .orderBy("label", "amount_range") \
    .show()

+-----+------+------------------+------------------+
|label|    nb|               avg|               std|
+-----+------+------------------+------------------+
|    1|   400|125.60019999999999|258.35134450196875|
|    0|227645|  88.3355424454762| 249.5706177774838|
+-----+------+------------------+------------------+

+-------+
| Amount|
+-------+
|2125.87|
|1504.93|
|1402.16|
|1389.56|
|1354.25|
| 1335.0|
|1218.89|
| 996.27|
| 925.31|
| 829.41|
+-------+

+-----+------------+------+
|label|amount_range| count|
+-----+------------+------+
|    0|        0-10| 77750|
|    0|      10-100|104034|
|    0|    100-1000| 43417|
|    0|       1000+|  2444|
|    1|        0-10|   203|
|    1|      10-100|    88|
|    1|    100-1000|   102|
|    1|       1000+|     7|
+-----+------------+------+



In [15]:
# b) Statistiques sur le montant (Section 3.3 & 3.4)
# Comme tu as un vecteur features, on extrait le montant
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col, avg, stddev, min, max, sum

# Convertir le vecteur en tableau
df_array = train_df.withColumn("features_array", vector_to_array(col("features")))

# Extraire la colonne Amount (position 29 selon ton TP)
df_with_amount = df_array.withColumn("Amount", col("features_array").getItem(29))

# Créer une nouvelle vue
df_with_amount.createOrReplaceTempView("transactions_with_amount")

# Statistiques par classe (comme Section 3.4)
print("\n=== Statistiques Montant par classe ===")
spark.sql("""
    SELECT label,
           COUNT(*) as count,
           ROUND(AVG(Amount), 2) as avg_amount,
           ROUND(STDDEV(Amount), 2) as std_amount,
           ROUND(MIN(Amount), 2) as min_amount,
           ROUND(MAX(Amount), 2) as max_amount,
           ROUND(SUM(Amount), 2) as total_amount
    FROM transactions_with_amount
    GROUP BY label
    ORDER BY label
""").show()


=== Statistiques Montant par classe ===
+-----+------+----------+----------+----------+----------+-------------+
|label| count|avg_amount|std_amount|min_amount|max_amount| total_amount|
+-----+------+----------+----------+----------+----------+-------------+
|    0|227645|     88.34|    249.57|       0.0|  25691.16|2.010914456E7|
|    1|   400|     125.6|    258.35|       0.0|   2125.87|     50240.08|
+-----+------+----------+----------+----------+----------+-------------+



In [16]:
print("\n=== Transactions suspectes (montant élevé) ===")
# Filtrer les transactions frauduleuses avec montant > 1000
spark.sql("""
    SELECT label, Amount, COUNT(*) as count
    FROM transactions_with_amount
    WHERE label = 1 AND Amount > 1000
    GROUP BY label, Amount
    ORDER BY Amount DESC
    LIMIT 10
""").show()


=== Transactions suspectes (montant élevé) ===
+-----+-------+-----+
|label| Amount|count|
+-----+-------+-----+
|    1|2125.87|    1|
|    1|1504.93|    1|
|    1|1402.16|    1|
|    1|1389.56|    1|
|    1|1354.25|    1|
|    1| 1335.0|    1|
|    1|1218.89|    1|
+-----+-------+-----+



In [22]:
print("\n📊 HISTOGRAMME : Distribution des montants par classe")

# Utiliser df_with_amount que tu as déjà créé
df_with_amount.createOrReplaceTempView("transactions_with_amount")

# Histogramme simple
histogram_simple = spark.sql("""
SELECT 
  CASE 
    WHEN Amount < 10 THEN '0-10'
    WHEN Amount < 50 THEN '10-50'
    WHEN Amount < 100 THEN '50-100'
    WHEN Amount < 500 THEN '100-500'
    WHEN Amount < 1000 THEN '500-1000'
    ELSE '1000+'
  END as plage_montant,
  label as classe,
  COUNT(*) as nb_transactions,
  ROUND(AVG(Amount), 2) as moyenne_montant
FROM transactions_with_amount
GROUP BY 
  CASE 
    WHEN Amount < 10 THEN '0-10'
    WHEN Amount < 50 THEN '10-50'
    WHEN Amount < 100 THEN '50-100'
    WHEN Amount < 500 THEN '100-500'
    WHEN Amount < 1000 THEN '500-1000'
    ELSE '1000+'
  END,
  label
ORDER BY 
  CASE 
    WHEN plage_montant = '0-10' THEN 1
    WHEN plage_montant = '10-50' THEN 2
    WHEN plage_montant = '50-100' THEN 3
    WHEN plage_montant = '100-500' THEN 4
    WHEN plage_montant = '500-1000' THEN 5
    ELSE 6
  END,
  label
""")

print("Distribution des montants par classe:")
histogram_simple.show()


📊 HISTOGRAMME : Distribution des montants par classe
Distribution des montants par classe:
+-------------+------+---------------+---------------+
|plage_montant|classe|nb_transactions|moyenne_montant|
+-------------+------+---------------+---------------+
|         0-10|     0|          77750|            3.7|
|         0-10|     1|            203|           1.73|
|        10-50|     0|          73973|          25.13|
|        10-50|     1|             47|          26.68|
|       50-100|     0|          30061|          71.81|
|       50-100|     1|             41|          87.31|
|      100-500|     0|          38250|         208.45|
|      100-500|     1|             79|         238.58|
|     500-1000|     0|           5167|         676.51|
|     500-1000|     1|             23|         690.26|
|        1000+|     0|           2444|        1773.93|
|        1000+|     1|              7|        1475.81|
+-------------+------+---------------+---------------+



In [23]:
print("\n📊 HISTOGRAMME 2D : MONTANT vs CLASSE DE FRAUDE")

# Créer un histogramme 2D avec groupBy
histogram_2d = spark.sql("""
SELECT 
  -- Axe X : Plages de montant
  CASE 
    WHEN Amount < 10 THEN '0-10'
    WHEN Amount < 50 THEN '10-50'
    WHEN Amount < 100 THEN '50-100'
    WHEN Amount < 500 THEN '100-500'
    WHEN Amount < 1000 THEN '500-1000'
    ELSE '1000+'
  END as montant_plage,
  
  -- Axe Y : Classe (0=normal, 1=fraude)
  label as classe,
  
  -- Valeur : Nombre de transactions
  COUNT(*) as nombre_transactions,
  
  -- Statistiques supplémentaires
  ROUND(AVG(Amount), 2) as montant_moyen,
  ROUND(MIN(Amount), 2) as montant_min,
  ROUND(MAX(Amount), 2) as montant_max
  
FROM transactions_with_amount
GROUP BY 
  CASE 
    WHEN Amount < 10 THEN '0-10'
    WHEN Amount < 50 THEN '10-50'
    WHEN Amount < 100 THEN '50-100'
    WHEN Amount < 500 THEN '100-500'
    WHEN Amount < 1000 THEN '500-1000'
    ELSE '1000+'
  END,
  label
ORDER BY 
  CASE 
    WHEN montant_plage = '0-10' THEN 1
    WHEN montant_plage = '10-50' THEN 2
    WHEN montant_plage = '50-100' THEN 3
    WHEN montant_plage = '100-500' THEN 4
    WHEN montant_plage = '500-1000' THEN 5
    ELSE 6
  END,
  classe
""")

print("Histogramme 2D - Distribution Montant vs Classe:")
histogram_2d.show()


📊 HISTOGRAMME 2D : MONTANT vs CLASSE DE FRAUDE
Histogramme 2D - Distribution Montant vs Classe:
+-------------+------+-------------------+-------------+-----------+-----------+
|montant_plage|classe|nombre_transactions|montant_moyen|montant_min|montant_max|
+-------------+------+-------------------+-------------+-----------+-----------+
|         0-10|     0|              77750|          3.7|        0.0|       9.99|
|         0-10|     1|                203|         1.73|        0.0|       9.99|
|        10-50|     0|              73973|        25.13|       10.0|      49.99|
|        10-50|     1|                 47|        26.68|       10.7|      45.64|
|       50-100|     0|              30061|        71.81|       50.0|      99.99|
|       50-100|     1|                 41|        87.31|       50.0|      99.99|
|      100-500|     0|              38250|       208.45|      100.0|     499.99|
|      100-500|     1|                 79|       238.58|      101.5|     489.71|
|     500-10

In [25]:
# 🚀 COMMANDE RAPIDE 2D

print("\n📊 HISTOGRAMME 2D RAPIDE")

quick_2d = spark.sql("""
SELECT 
  -- Dimension X: Montant (3 catégories)
  CASE 
    WHEN Amount < 100 THEN 'Petit (<100)'
    WHEN Amount < 1000 THEN 'Moyen (100-1000)'
    ELSE 'Grand (>1000)'
  END as taille,
  
  -- Dimension Y: Type
  CASE 
    WHEN label = 1 THEN 'FRAUDE'
    ELSE 'NORMAL'
  END as type,
  
  -- Valeur
  COUNT(*) as nombre,
  ROUND(AVG(Amount), 2) as moyenne
  
FROM transactions_with_amount
GROUP BY 
  CASE 
    WHEN Amount < 100 THEN 'Petit (<100)'
    WHEN Amount < 1000 THEN 'Moyen (100-1000)'
    ELSE 'Grand (>1000)'
  END,
  CASE 
    WHEN label = 1 THEN 'FRAUDE'
    ELSE 'NORMAL'
  END
ORDER BY 
  CASE 
    WHEN taille = 'Petit (<100)' THEN 1
    WHEN taille = 'Moyen (100-1000)' THEN 2
    ELSE 3
  END,
  CASE 
    WHEN type = 'FRAUDE' THEN 1
    ELSE 2
  END
""")

print("Histogramme 2D simple:")
quick_2d.show()


📊 HISTOGRAMME 2D RAPIDE
Histogramme 2D simple:
+----------------+------+------+-------+
|          taille|  type|nombre|moyenne|
+----------------+------+------+-------+
|    Petit (<100)|FRAUDE|   291|  17.82|
|    Petit (<100)|NORMAL|181784|  23.68|
|Moyen (100-1000)|FRAUDE|   102| 340.43|
|Moyen (100-1000)|NORMAL| 43417| 264.16|
|   Grand (>1000)|FRAUDE|     7|1475.81|
|   Grand (>1000)|NORMAL|  2444|1773.93|
+----------------+------+------+-------+

